In [ ]:
# Import python packages
import pandas as pd
from simple_salesforce import Salesforce,SalesforceMalformedRequest
import json
import datetime
import base64

In [ ]:
# Create an active session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Get sf_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.ARVINDR;
CREATE OR REPLACE function sf_hcorp_api_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SALESFORCE_APIS_ACCESS_INTEGRATION)
SECRETS = ('cred' = PYTHON_SALESFORCE_PRD_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sf_hcorp_api_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.ARVINDR.sf_hcorp_api_secret()"
sf_token = session.sql(api_secret_query).collect()[0][0]

In [ ]:
-- Get sf_password from Snowflake secrets
use schema PYTHON_WORKLOADS.ARVINDR;
CREATE OR REPLACE function sf_hcorp_pass_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SALESFORCE_APIS_ACCESS_INTEGRATION)
SECRETS = ('cred' = PYTHON_SALESFORCE_PRD)
AS
$$
import _snowflake
def get_secret_1():
    username_password_object = _snowflake.get_username_password('cred');
    username_password_dictionary = {}
    username_password_dictionary["Username"] = username_password_object.username
    username_password_dictionary["Password"] = username_password_object.password
    return username_password_dictionary
$$;
grant usage on function sf_hcorp_pass_secret() to role svc_engineer_prd;

In [ ]:
# Store sf_password in variable
api_secret_query = "select PYTHON_WORKLOADS.ARVINDR.sf_hcorp_pass_secret()"
sf_pass = session.sql(api_secret_query).collect()[0][0]
sf_pass = sf_pass.replace("\'", "\"")
sf_creds = json.loads(sf_pass)
sf_password = sf_creds["Password"]
sf_uname = sf_creds["Username"]

In [ ]:
# Connect to Salesforce - PRD
sf = Salesforce (username=sf_uname, password=sf_password, security_token=sf_token) 

In [ ]:
select distinct 
    z.id as lead_id,
    o.id as Id,
    z.zi_c_location_id as location_id, 
    z.zi_c_company_id as company_id,
    o.ZI_LOCATION_ID__C as sfdc_location_id,
    o.ZI_COMPANY_ID__C as sfdc_company_id
from zoominfo.enrichment.api_results_history z
join salesforce.sfdc.lead l on z.id = l.id
left join 
    (select distinct 
        id, 
        original_lead_id__c,sfdc_lead_id__c,leadid__c,zi_location_id__c,zi_company_id__c,
        coalesce(original_lead_id__c,sfdc_lead_id__c,leadid__c) as opp_lead_id__c
    from salesforce.sfdc.opportunity where (zi_company_id__c is null or zi_location_id__c is null or zi_company_id__c=0 or zi_location_id__c=0) 
    )o 
    on l.id = o.opp_lead_id__c
where z.match_score >=4 and z.id_type='LEAD' and o.id is not null
;

In [ ]:
# Convert and format the retreived data into a dataframe
opportunity_df = cell8.to_pandas()
opportunity_df = opportunity_df.fillna('')
errorlist = []
failed_ids = []

In [ ]:
opportunity_df = opportunity_df[(opportunity_df.LOCATION_ID != '') | (opportunity_df.COMPANY_ID != '')]
opportunity_df = opportunity_df.rename(columns={'ID': 'Id','LOCATION_ID': 'ZI_Location_ID__c', 'COMPANY_ID': 'ZI_Company_ID__c'})
opportunity_final_df = opportunity_df[['Id','ZI_Location_ID__c','ZI_Company_ID__c']]
opportunity_json = opportunity_final_df.to_json(orient='records',date_format='iso')
update_data_list = json.loads(opportunity_json)
print(update_data_list)

In [ ]:
# Check if any entry in salesforce needs to be updated and update them
if len(update_data_list)>0:
    try:
        results=sf.bulk.Opportunity.update(update_data_list,batch_size=5000,use_serial=True)
        if results[0]['success']:
            pass
        else:
            errorlist.append(results[0]['errors'])
    except Exception as e:
        errorlist.append(str(e))
        pass
else:
    results=[]
    print('No new entries')

In [ ]:
if len(errorlist)==0:
    message2 = f'SFDC_ENRICHMENT_OPPORTUNITY Notebook ran successfully. Total rows updated in Opportunity object: {len(update_data_list)}'
    print(message2)
else:
    t1 = "".join(str(element) for element in errorlist)
    t2 = "".join(str(element) for element in failed_ids)
    message = f'Please check SFDC_ENRICHMENT_OPPORTUNITY Notebook, Error is: ' + t1 + '\n' + t2 + ''
    print(message)